In [1]:
#importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras

Using TensorFlow backend.


In [2]:
# Read data files
comments_attack = pd.read_csv('attack_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations_attack = pd.read_csv('attack_annotations.tsv',  sep = '\t')
comments_aggression = pd.read_csv('aggression_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations_aggression = pd.read_csv('aggression_annotations.tsv',  sep = '\t')
comments_toxicity = pd.read_csv('toxicity_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations_toxicity = pd.read_csv('toxicity_annotations.tsv',  sep = '\t')

In [3]:
# labels a comment if the majority of annoatators did so
labels_attack = annotations_attack.groupby('rev_id')['attack'].mean() > 0.5
labels_aggression = annotations_aggression.groupby('rev_id')['aggression'].mean() > 0.5
labels_toxicity = annotations_toxicity.groupby('rev_id')['toxicity'].mean() > 0.5


In [7]:
labels_aggression

rev_id
37675        False
44816        False
49851        False
89320        False
93890        False
102817       False
103624       False
111032       False
120283       False
128532       False
133562       False
138117       False
155243       False
177310       False
192579       False
201190       False
208009       False
249432       False
252031       False
268558       False
276906       False
286174       False
290598       False
294124       False
297866       False
317177       False
336654       False
344567       False
356383       False
358984       False
             ...  
699646005     True
699659494     True
699660419     True
699661020     True
699661834    False
699663770    False
699664687     True
699667660    False
699683891    False
699698850    False
699702006    False
699703322    False
699715740    False
699728036    False
699730832    False
699732149    False
699741197    False
699753082    False
699755057    False
699756053    False
699756185    False
69978

In [5]:
# join labels and comments
comments_attack['label'] = labels_attack*1.0
comments_aggression['label'] = labels_aggression*2.0
comments_toxicity['label'] = labels_toxicity*3.0

In [8]:
comments_aggression['label']
699646005    2.0
699659494    2.0
699660419    2.0

rev_id
37675        0.0
44816        0.0
49851        0.0
89320        0.0
93890        0.0
102817       0.0
103624       0.0
111032       0.0
120283       0.0
128532       0.0
133562       0.0
138117       0.0
155243       0.0
177310       0.0
192579       0.0
201190       0.0
208009       0.0
249432       0.0
252031       0.0
268558       0.0
276906       0.0
286174       0.0
290598       0.0
294124       0.0
297866       0.0
317177       0.0
336654       0.0
344567       0.0
356383       0.0
358984       0.0
            ... 
699646005    2.0
699659494    2.0
699660419    2.0
699661020    2.0
699661834    0.0
699663770    0.0
699664687    2.0
699667660    0.0
699683891    0.0
699698850    0.0
699702006    0.0
699703322    0.0
699715740    0.0
699728036    0.0
699730832    0.0
699732149    0.0
699741197    0.0
699753082    0.0
699755057    0.0
699756053    0.0
699756185    0.0
699780538    0.0
699813325    0.0
699820699    0.0
699822249    0.0
699848324    0.0
699851288    0.0
6998571

In [9]:
comments_attack = comments_attack.query("label == 1.0")
comments_aggression = comments_aggression.query("label == 2.0")
comments_toxicity= comments_toxicity.query("label == 3.0")

In [10]:
# Concatenation of the three data sets
dataframe = pd.concat([comments_attack, comments_aggression, comments_toxicity], axis = 0)
dataframe

,comment,year,logged_in,ns,sample,split,label
rev_id,,,,,,,
801279.0,Iraq is not good ===NEWLINE_TOKENNEWLINE_TO...,2003,False,article,random,train,1.0
2702703.0,NEWLINE_TOKENNEWLINE_TOKEN____NEWLINE_TOKENfuc...,2004,False,user,random,train,1.0
4632658.0,"i have a dick, its bigger than yours! hahaha",2004,False,article,blocked,train,1.0
6545332.0,NEWLINE_TOKENNEWLINE_TOKEN== renault ==NEWLINE...,2004,True,user,blocked,train,1.0
6545351.0,NEWLINE_TOKENNEWLINE_TOKEN== renault ==NEWLINE...,2004,True,user,blocked,test,1.0
7977970.0,"34, 30 Nov 2004 (UTC)NEWLINE_TOKENNEWLINE_TOKE...",2004,True,article,random,train,1.0
8359431.0,`NEWLINE_TOKENNEWLINE_TOKEN::You are not worth...,2004,True,user,blocked,train,1.0
8724028.0,"Yes, complain to your rabbi and then go shoot ...",2004,True,user,blocked,test,1.0
8845700.0,NEWLINE_TOKENNEWLINE_TOKENi am using the sandb...,2004,False,user,blocked,train,1.0


In [42]:
# remove newline and tab tokens
import re
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.lower())
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('.,[^a-zA-z0-9\s]','',x)))

In [44]:
dataframe

,comment,year,logged_in,ns,sample,split,label
rev_id,,,,,,,
801279.0,iraq is not good usa is bad,2003,False,article,random,train,1.0
2702703.0,____ fuck off you little asshole if you want...,2004,False,user,random,train,1.0
4632658.0,i have a dick its bigger than yours hahaha,2004,False,article,blocked,train,1.0
6545332.0,renault you sad little bpy for driving a ...,2004,True,user,blocked,train,1.0
6545351.0,renault you sad little bo for driving a r...,2004,True,user,blocked,test,1.0
7977970.0,34 30 nov 2004 utc because you like to accuse...,2004,True,article,random,train,1.0
8359431.0,` you are not worth the effort you are arguin...,2004,True,user,blocked,train,1.0
8724028.0,yes complain to your rabbi and then go shoot s...,2004,True,user,blocked,test,1.0
8845700.0,i am using the sandbox ass wipe,2004,False,user,blocked,train,1.0


In [45]:
train_comments = dataframe.query("split=='train'")
valid_comments = dataframe.query("split=='test'")

In [52]:
train_comments.iloc[100]['comment']


'   aquafag   if you continue to vandalize my talk page you will be blocked from living on the earth wam   thank you  '

In [53]:
print('train comments length: ',len(train_comments))
print('test comments length: ',len(valid_comments))

train comments length:  26143
test comments length:  8789


In [54]:
# split the dataset into training and validation datasets 
train_x, valid_x = train_comments['comment'], valid_comments['comment'], 
train_y, valid_y = train_comments['label'], valid_comments['label']                                                       
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)
print(train_y.shape)

(26143,)


In [55]:
type(train_x)

pandas.core.series.Series

In [67]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(dataframe['comment'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [71]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(dataframe['comment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
tfidf_vect_ngram.fit(dataframe['comment'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(dataframe['comment'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [72]:
xtrain_tfidf_ngram.shape

(26143, 5000)

In [80]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(dataframe['comment'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [65]:
def train_model(classifier, xtrain, ytrain, xvalid, yvalid):
    
    #predictions = predictions.argmax(axis=-1)
   
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)
        
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average=None)
    return accuracy, f1score

In [73]:
# Naive Bayes on Count Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count, valid_y)
print("NB, Count Vectors:   accuracy: %s      f1 score: %s"% (accuracy,f1score))

NB, Count Vectors:   accuracy: 0.3408806462623734      f1 score: [0.19035337 0.27935144 0.44558378]


In [74]:
# Naive Bayes on Count Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count, valid_y)
print("NB, Count Vectors:   accuracy: %s      f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Word Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Character Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

NB, Count Vectors:   accuracy: 0.3408806462623734      f1 score: [0.19035337 0.27935144 0.44558378]
NB, WordLevel TF-IDF:   accuracy: 0.2899078393446353     f1 score: [0.07061713 0.29476966 0.37018837]
NB, N-Gram Vectors:   accuracy: 0.31107065650244625     f1 score: [0.06279734 0.27434634 0.41784192]
NB, CharLevel Vectors:   accuracy: 0.3168733644328137   f1 score: [0.08637874 0.32567707 0.40128068]


In [75]:
# Linear Classifier on Count Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count, valid_y)
print("LR, Count Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("LR, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("LR, N-Gram Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

C:\Users\Silver\AppData\Roaming\Python\Python35\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Silver\AppData\Roaming\Python\Python35\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Silver\AppData\Roaming\Python\Python35\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LR, Count Vectors:   accuracy: 0.3520309477756286   f1 score: [0.00359842 0.16787942 0.50515935]
LR, WordLevel TF-IDF:   accuracy: 0.28478780293548756   f1 score: [0.13087157 0.28394186 0.35405307]
LR, N-Gram Vectors:   accuracy: 0.3043577198771191   f1 score: [0.10925819 0.25998831 0.40539637]
LR, CharLevel Vectors:   accuracy: 0.29775856183866195   f1 score: [0.11820463 0.31378548 0.36556487]


In [ ]:
# SVM Classifier on Count Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print("SVM, Count Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# SVM Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print("SVM, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# SVM on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# SVM Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("SVM, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

In [76]:
# Feed forward NN with 1 hidden layer
def model_FF(xtrain, ytrain, xvalid, yvalid, hidden_size, epochs =1):
    # create input layer 
    input_layer = layers.Input((xtrain.shape[1], ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(hidden_size, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(3, activation="softmax")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    classifier.fit(xtrain, ytrain,
                  batch_size=256,
                  epochs=epochs,
                  shuffle = True)
    # scores of the classifier
    scores = classifier.evaluate(xvalid, yvalid, verbose=0)
    return scores

In [77]:
# convert to one_hot
train_y_onehot = keras.utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.to_categorical(valid_y, 3)
train_y_onehot.shape

(26143, 3)

In [78]:

# NN Classifier on Count Vectors
scores = model_FF(xtrain_count, train_y_onehot, xvalid_count, valid_y_onehot, 100)
print("NN, Count Vectors accuracy: ", scores[1])

# NN Classifier on Word Level TF IDF Vectors
scores = model_FF(xtrain_tfidf, train_y_onehot, xvalid_tfidf, valid_y_onehot, 100)
print("NN, Count Vectors accuracy: ", scores[1])

# NN Classifier on Ngram Level TF IDF Vectors
scores = model_FF(xtrain_tfidf_ngram, train_y_onehot, xvalid_tfidf_ngram, valid_y_onehot, 100)
print("NN, Count Vectors accuracy: ", scores[1])

# NN Classifier on Count Vectors
scores = model_FF(xtrain_tfidf_ngram_chars, train_y_onehot, xvalid_tfidf_ngram_chars, valid_y_onehot, 100)
print("NN, Count Vectors accuracy: ", scores[1])

Epoch 1/1
26143/26143 [==============================] - 3s 100us/step - loss: 1.7157 - acc: 0.3327
NN, Count Vectors accuracy:  0.3351917169460028
Epoch 1/1
26143/26143 [==============================] - 2s 95us/step - loss: 1.0996 - acc: 0.3460: 3
NN, Count Vectors accuracy:  0.32108317218769233
Epoch 1/1
26143/26143 [==============================] - 2s 86us/step - loss: 1.0998 - acc: 0.3429
NN, Count Vectors accuracy:  0.34235976789168276
Epoch 1/1
26143/26143 [==============================] - 4s 162us/step - loss: 1.0990 - acc: 0.3440 2s - loss: 1.0989  - ETA: 1s - loss: 1.
NN, Count Vectors accuracy:  0.3456593469109114


In [82]:
def cnn(xtrain, ytrain, xvalid, yvalid, epochs = 10):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=epochs)
    scores = model.evaluate(xvalid, yvalid, verbose=0)
    return scores

In [83]:
scores = cnn(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("CNN, Word Embeddings acuuracy: ",scores[1])

Epoch 1/10
26143/26143 [==============================] - 42s 2ms/step - loss: 1.1002 - acc: 0.3477
Epoch 2/10
26143/26143 [==============================] - 37s 1ms/step - loss: 1.0974 - acc: 0.3529
Epoch 3/10
26143/26143 [==============================] - 37s 1ms/step - loss: 1.0967 - acc: 0.3580
Epoch 4/10
26143/26143 [==============================] - 36s 1ms/step - loss: 1.0957 - acc: 0.3622
Epoch 5/10
26143/26143 [==============================] - 36s 1ms/step - loss: 1.0949 - acc: 0.3627
Epoch 6/10
26143/26143 [==============================] - 36s 1ms/step - loss: 1.0937 - acc: 0.3668
Epoch 7/10
26143/26143 [==============================] - 37s 1ms/step - loss: 1.0917 - acc: 0.3733
Epoch 8/10
26143/26143 [==============================] - 36s 1ms/step - loss: 1.0897 - acc: 0.3749
Epoch 9/10
26143/26143 [==============================] - 36s 1ms/step - loss: 1.0873 - acc: 0.3791
Epoch 10/10
26143/26143 [==============================] - 36s 1ms/step - loss: 1.0828 - acc: 0.3880

In [56]:
def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 3):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.LSTM(100))(embedding_layer)
    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=3)
    
    scores = model.evaluate(xvalid, yvalid, verbose=0)
    return scores

In [57]:
scores = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("LSTM, Word Embeddings:  accuracy: ", scores[1])

Epoch 1/3
 6144/26143 [======>.......................] - ETA: 2:30 - loss: 0.6373 - acc: 0.6667

KeyboardInterrupt: 